ITU - Kariyer Zirvesi - Generative AI Workshop

Bu notebook, Retrieval-Augmented Generation (RAG) modelinin basit bir implementasyonunu sunmaktadır. RAG, büyük dil modelleri ile bilgi arama süreçlerini entegre ederek daha doğru ve bağlamsal yanıtlar üretmeye yardımcı olan bir yöntemdir. Bu notebook'ta, bir bilgi veritabanı (corpus) kullanarak sorulara cevaplar üretmek için RAG mimarisinin temel adımlarını keşfedeceksiniz.

Ana Adımlar:

Veri Hazırlığı: Kullanıcıdan gelen soruları anlamak için uygun veri kümesi sağlanır. Bilgi Arama: RAG modeli, belirli bir sorguya yönelik bilgiyi veritabanından (retriever) alır. Cevap Üretme: Bilgi arama işleminden sonra, model bu bilgiyi kullanarak soruya daha anlamlı ve doğru cevaplar üretir. Bu implementasyon, doğal dil işleme (NLP) ve dil modelleri hakkında daha derinlemesine bilgi edinmek isteyenler için başlangıç seviyesinde bir rehberdir.

Not: Notebook'ta kullanılan yöntemler ve açıklamalar, özellikle başlangıç seviyesindeki kullanıcılar için sadeleştirilmiştir. Başka projelere entegre edebilmeniz için temel yapıyı anlamanıza yardımcı olacaktır.

# RAG Application
Bu bölümde temel bir RAG uygulaması gerçekleştireceğiz. En basit şekilde bir bankanın açık olarak sunduğu finansal veri raporu ile nasıl konuşabileceğimizi göstereceğiz.

## Step 1 - Paket ve Enviroment Hazırlığı

Bu adımda RAG Uygulamamız için gerekli olan paketleri yükleyip, yüklenen paketlerden ihtiyacımız olan sınıfları kodumuza dahil edeceğiz.

RAG Uygulamasını temel anlamda ayağa kaldırabilmek için langchain, faiss, openai ve pdf loader kütüphanelerine ihtiyacımız olacak. Notebook'a bunları yüklüyoruz.

In [1]:
!pip install langchain
!pip install openai
!pip install tiktoken
!pip install faiss-gpu
!pip install faiss-cpu
!pip install langchain_experimental
!pip install "langchain[docarray]"
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.0 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement faiss-gpu (from versions: none)
ERROR: No matching distribution found for faiss-gpu
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 62.9 MB/s eta 0:00:00
  Using cached langchain_experimental-0.3.4-py3-none-any.whl.metadata (1.7 kB)
  Using cached langchain_community-0.3.17-py3-none-any.whl.metadata (2.4 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached pydantic_settings-2.7.1-py3-none-any.whl.metadata (3.5 kB)
  Using cached httpx_sse-0.4.0-py3-none-any.whl.metadata (9.0 kB)
  Using cached marshmallow-3.26.1-py3-none-any.whl.metadata (7.3 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
  Using cached mypy_extensions-1.0.0-py3-none-any.whl.metadata (1.1 kB)
Using cached langchain_experimental-0.3.4-py3-no

In [2]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader,PyPDFLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.indexes import VectorstoreIndexCreator
from langchain_experimental.agents.agent_toolkits.csv.base import create_csv_agent
from langchain.agents.agent_types import AgentType
from langchain.memory import ConversationBufferMemory
from langchain.text_splitter import RecursiveCharacterTextSplitter
import tiktoken

RAG Uygulamamızın LLM'i ve Embedding modelini OpenAI'dan kullanacağız. Bu sebeple API-KEY'imizi sistemimize tanımlamamız gerekiyor.

In [26]:
import os

# Prompt the user for their OpenAI API key
api_key = input("Please enter your OpenAI API key: ")

# Set the API key as an environment variable
os.environ["OPENAI_API_KEY"] = api_key

# Optionally, check that the environment variable was set correctly
print("OPENAI_API_KEY has been set!")


Please enter your OpenAI API key: test
OPENAI_API_KEY has been set!


Step 2 - Dökümanı İşleme

## Bu bölümde bahse konu finansal veri dökümanını sistemimize entegre edeceğiz. Bu bağlamda öncelikle pdf'i buffer edeceğiz. Daha sonrasında bunu chunklara ayırıp vectorize edeceğiz.

In [5]:
file_path = '/content/ISBTR-31.12.2024-Konsolide Olmayan Finansal Rapor.pdf'
loader = PyPDFLoader(file_path)
data = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
data = text_splitter.split_documents(data)

In [ ]:
data

In [6]:
# Create vector store
embeddings = OpenAIEmbeddings(openai_api_key=os.environ["OPENAI_API_KEY"], disallowed_special=(), model="text-embedding-3-large")
vectorstore = FAISS.from_documents(data, embedding=embeddings)

<ipython-input-6-1b991a0048e8>:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(openai_api_key=os.environ["OPENAI_API_KEY"], disallowed_special=(), model="text-embedding-3-large")


## Step 3 - Generation
Bu bölümde dökümandan elde ettiğimiz verilerle LLM yardımıyla üretim yapacağız.

In [11]:
# Create conversation chain
llm = ChatOpenAI(temperature=0.7, model_name="gpt-4o")
memory = ConversationBufferMemory(
memory_key='chat_history', return_messages=True)
conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        chain_type="stuff",
        retriever=vectorstore.as_retriever(),
        memory=memory
        )

In [8]:
query = "Hisse Senedi Yatırımları ne kadar ?"
result = conversation_chain({"question": query})
answer = result["answer"]
answer

<ipython-input-8-c2418405968c>:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = conversation_chain({"question": query})


'Hisse Senedi Yatırımları 139.830.305 olarak verilmiştir.'

In [9]:
query = "Bölgesel Yönetimlerden veya Yerel Yönetimlerden Alacaklar Kredi Dönüşüm Oranı ve Kredi Riski Azaltımından Önce Alacak Tutarı bilanço dışı ne kadar ?"
result = conversation_chain({"question": query})
answer = result["answer"]
answer

"Bölgesel Yönetimlerden veya Yerel Yönetimlerden Alacaklar Kredi Dönüşüm Oranı ve Kredi Riski Azaltımından Önce Alacak Tutarı bilanço dışı 4.439'dur."

In [10]:
query = "Cari Dönem Bilanço Dışı Tutarlar toplam ne kadardır?"
result = conversation_chain({"question": query})
answer = result["answer"]
answer

"Cari Dönem Bilanço Dışı Borçlar toplamı 437.467.763'tür."

# Single Agent Application

Bu bölümde daha önce yaptığımız RAG yaklaşımını bir agent'a çevireceğiz. Bu agent içinde birden fazla fonksiyonu barındırıp çevresel dinlemekler yapabilecek.

## Step - 1 RAG Tool Tanımlama

Bu bölümde daha önce yapmış olduğumuz RAG işlemini bir araç olarak agent'ımıza tanımlayacağız.

In [13]:
from langchain.tools.retriever import create_retriever_tool

In [14]:
retriever_tool = create_retriever_tool (vectorstore.as_retriever(), "Is-Bank-Search", "Search for financial information about İs Bankası. For any questions about İs Bankası, you must use this tool")


## Step - 2 Web Search Tool Implementasyonu

Bu bölümde Agent'ın dışarı ile bağlantısını kuracak bir search toolu yaracağız.

In [24]:
import os

tavily_api_key = input("Please enter your Tavily API key: ")
os.environ['TAVILY_API_KEY'] = tavily_api_key

Please enter your Tavily API key: test


In [25]:
from langchain_community.tools.tavily_search import TavilySearchResults
search = TavilySearchResults()

## Step - 3 Generation

Ürettiğimiz iki tool'u da bir agent'a bağlayıp onun üzerinden iletim yapacağız.

In [16]:
tools = [retriever_tool, search]

In [17]:
!pip install langchainhub
!pip install langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 3.5 MB/s eta 0:00:00


In [18]:
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [19]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(api_key=os.environ["OPENAI_API_KEY"],temperature=0.7, model_name="gpt-4o")

In [20]:
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor

In [21]:
agent = create_openai_functions_agent(llm,tools,prompt)
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)

In [22]:
agent_executor.invoke({"input":"İşbankasının Hisse Senedi Yatırımları ne kadar ?"})



> Entering new AgentExecutor chain...

Invoking: `Is-Bank-Search` with `{'query': 'İş Bankası hisse senedi yatırımları'}`


Kurumsal ve Ticari Krediler Takip 
Mehmet Celayir 
Destek Hizmetleri, İnsan Kaynakları Yönetimi, İnşaat ve Gayrimenkul Yönetimi, Yetenek 
Yönetimi, Tüketici İlişkileri Koordinasyon Görevlisi, Operasyonel Risk Komitesi ve 
Sürdürülebilirlik Komitesi Üyesi 
Mehmet Türk Finansal Yönetim, Strateji ve Kurumsal Performans Yönetimi, Bilgi Sistemleri Strateji Komitesi 
ve Sürdürülebilirlik Komitesi Üyesi 
 
Yönetim Kurulu ve Denetim Komitesi Üyeleri, Genel Müdür ve Genel Müdür Yardımcılarının Banka’da sahip olduğu paylar çok 
önemsiz seviyededir. 
 
IV. Banka’da Nitelikli Pay Sahibi Olan Kişi ve Kuruluşlara İlişkin Açıklama 
 
Ad Soyad/Ticaret Unvanı Pay Tutarları  Pay Oranları Ödenmiş Paylar Ödenmemiş Paylar 
T. İş Bankası A.Ş. Mensupları Munzam Sosyal 
Güvenlik ve Yardımlaşma Sandığı Vakfı 9.647.873 %38,59 9.647.873  
Atatürk Hisseleri (CHP) 7.023.013  %28,09 7.023.01

{'input': 'İşbankasının Hisse Senedi Yatırımları ne kadar ?',
 'output': "İş Bankası'nın hisse senedi yatırımları ile ilgili bazı detaylar şu şekildedir:\n\n1. **Borsada İşlem Gören Hisse Senedi Yatırımları:**\n   - Mali bağlı ortaklıklar için bilanço değeri 62.257.824 TL ve gerçeğe uygun değer 177.836.970 TL'dir.\n   - Mali olmayan bağlı ortaklıklar için bilanço değeri 80.680.556 TL ve gerçeğe uygun değer 128.694.810 TL'dir.\n\n2. **Borsada İşlem Görmeyen İştirakler:**\n   - Mali iştirakler için bilanço değeri 554.844 TL'dir.\n   - Mali olmayan iştirakler için bilanço değeri 75.248 TL'dir.\n\nBu bilgiler İş Bankası'nın mali raporlarında belirtilen tutarlardır ve bankanın iştirak ve bağlı ortaklık niteliğindeki hisse senedi yatırımlarını içermektedir. Banka, iştiraklerini ve bağlı ortaklıklarını muhasebe uygulamaları çerçevesinde değerlendirmekte ve bunların piyasa ile karşılaştırmalı değerlerini raporlamaktadır."}

In [23]:
agent_executor.invoke({"input":"İTÜ Kariyer Zirvesi 2025'te hangi tarihler arasında düzenlenecek ?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'İTÜ Kariyer Zirvesi 2025 tarihleri'}`


[{'url': 'http://etkinlik.itu.edu.tr/EtkinlikDetay.aspx?etkinlik=2559', 'content': 'İTÜ Kariyer Zirvesi 2025 · Tarih. 17.2.2025 10:00 - 21.2.2025 17:00 · Mekan. Merkezi Derslik Binası, Ayazağa Kampüsü · Etkinlik Kategorisi. Profesyonel Yaşam. İTÜ'}, {'url': 'https://etkinlik.itu.edu.tr/', 'content': "İTÜ Kariyer Zirvesi (İKZ'25), 17-21 Şubat 2025 tarihleri arasında düzenlenecek ve farklı sektörlerden firmalar stant açarak İTÜ öğrencileriyle buluşacak."}, {'url': 'https://ari24.com/etkinlik/21-itu-kariyer-zirvesi-7575', 'content': '21. İTÜ Kariyer Zirvesi ; Başlangıç Tarihi, 17 Şubat 2025 10:00 · Bitiş Tarihi, 21 Şubat 2025 17:00. Konum, İTÜ · Konum, İTÜ · Etkinlik Türü, Zirve'}, {'url': 'https://www.instagram.com/itukariyer/p/DGKdXUOMI4N/', 'content': "... kariyer etkinliklerinden biri olan İTÜ Kariyer Zirvesi 2025 (İKZ'25) heyecanı başlıyor. ⏳ 17-21 

{'input': "İTÜ Kariyer Zirvesi 2025'te hangi tarihler arasında düzenlenecek ?",
 'output': 'İTÜ Kariyer Zirvesi 2025, 17-21 Şubat 2025 tarihleri arasında düzenlenecek.'}